In [31]:
import os
import cv2
import imutils
import math
import numpy as np
from tqdm import tqdm

In [32]:
def disp_img(img):
    cv2.imshow("Image", img)
    cv2.waitKey(0)

def normalize_img(img):
    img = (img-np.min(img))/(np.max(img)-np.min(img)) * 255
    img = img.astype("uint8")
    return img

def write_img(name, img):
    cv2.imwrite("Output_Images//Sobel//Edge_Detected_%s" % name, img)
    
def apply_gaussian_blur(img, sigma=1):
    kernel_x = np.array([[1, 4, 7, 4 ,1],
              [4, 16,26,16,4],
              [7, 26,41,26,7],
              [4, 16,26,16,4],
              [1, 4, 7, 4 ,1]], dtype='float')
    kernel_y = kernel_x.T
    kernel = (1/(2*np.pi*(sigma**2))) * (np.exp(-(np.square(kernel_x) + np.square(kernel_y)) / (2*np.square(sigma))))
    kernel = kernel / np.sum(kernel)
    
    blurred = np.zeros(img.shape)
    for i in tqdm(range(2 , img.shape[0] - 2)):
        for j in range(2, img.shape[1] - 2):
            blurred[i, j] = np.sum(img[i-2:i+3, j-2:j+3]*kernel)
    return blurred
    
def run_sobel_filter(img):
    window_y = np.array([[-1, 0 ,1], [-2, 0, 2], [-1, 0 ,1]])
    window_x = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    sobel_x = np.zeros(img.shape)
    sobel_y = np.zeros(img.shape)
    
    img = np.pad(img, 1, mode='edge')
    for i in tqdm(range(1 , img.shape[0] - 1)):
        for j in range(1, img.shape[1] - 1):
            img_blob = img[i-1:i+2, j-1:j+2]
            sobel_x[i-1, j-1] = np.sum(img_blob*window_x)
            sobel_y[i-1, j-1] = np.sum(img_blob*window_y)
            
            
    return sobel_x, sobel_y

def perform_nonmax_suppression(img):
    for i in range(2, img.shape[0] - 2):
        for j in range(2, img.shape[1] - 2):
            blob = img[i-2:i+3, j-2:j+3]
            

In [33]:
for name in os.listdir("Input_Images"):
    final_path = "Input_Images//" + name
    print("#### Running Edge detection on Image : %s ####" % name)
    img = cv2.imread(final_path)
    grey_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #grey_img = apply_gaussian_blur(grey_img, 1)
    grey_img = cv2.GaussianBlur(grey_img, (5, 5), 0)
        
    sobel_x, sobel_y = run_sobel_filter(grey_img)
    print("Sobel generated")
    final_sobel = np.sqrt(np.square(sobel_x) + np.square(sobel_y))
    final_sobel = final_sobel.astype("uint8")
    
    radian_img = np.arctan2(sobel_y // sobel_x)
    #radian_img = np.rad2deg(radian_img)
    radian_img = radian_img.astype("uint8")
    print(radian_img.shape)
    dark = np.zeros(radian_img.shape).astype("uint8")
    color_img = cv2.merge([radian_img, dark, 255 - radian_img])
    
    #color_img = cv2.cvtColor(radian_img, cv2.COLOR_GRAY2RGB)
    
    write_img(name, final_sobel)
    write_img("_With_Direction_" + name, color_img)
    
    
    print("Image created")


#### Running Edge detection on Image : BedSheet.jpeg ####


100%|██████████| 2016/2016 [00:59<00:00, 34.17it/s]


Sobel generated


C:\Users\nikhi\AppData\Local\Temp/ipykernel_22064/1632394700.py:14: RuntimeWarning: divide by zero encountered in floor_divide
  radian_img = np.arctan2(sobel_y // sobel_x)
C:\Users\nikhi\AppData\Local\Temp/ipykernel_22064/1632394700.py:14: RuntimeWarning: invalid value encountered in floor_divide
  radian_img = np.arctan2(sobel_y // sobel_x)


ValueError: invalid number of arguments